In [1]:
from datetime import datetime
import os
import re
import glob
import requests 
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np
import folium

In [2]:
link = 'https://www.worldometers.info/coronavirus/'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
thead = soup.find_all('thead')[-1]
head = thead.find_all('tr')
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

In [4]:
head_rows = []
body_rows = []
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [5]:
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         
df_bs.head(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,,\nNorth America\n,"3,628,797","+7,809","178,674",+945,"1,648,713","+5,170","1,801,410","18,601",,,,,,North America,\n,,
1,,\nSouth America\n,"2,614,931","+1,036","96,832",+54,"1,717,350","+46,162","800,749","13,639",,,,,,South America,\n,,
2,,\nAsia\n,"2,704,095","+23,834","64,965",+523,"1,841,893","+20,421","797,237","19,572",,,,,,Asia,\n,,
3,,\nEurope\n,"2,513,905","+9,228","194,790",+242,"1,463,217","+11,882","855,898","5,456",,,,,,Europe,\n,,
4,,\nAfrica\n,"511,949",+779,"12,026",+11,"248,751","+1,017","251,172",965,,,,,,Africa,\n,,


In [6]:
df_bs = df_bs.iloc[8:, :-3].reset_index(drop=True)
df_bs = df_bs.drop('#', axis=1)
df_bs.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent
0,USA,"3,097,538",+454,"133,991",+19,"1,355,524",+661,"1,608,023","15,371","9,357",405,"38,806,121","117,225","331,039,330",North America
1,Brazil,"1,674,655",,"66,868",,"1,117,922","+45,693","489,865","8,318","7,878",315,"4,359,978","20,509","212,587,032",South America
2,India,"746,506","+3,025","20,684",+31,"458,618","+1,560","267,204","8,944",541,15,"10,473,771","7,588","1,380,233,788",Asia
3,Russia,"700,792","+6,562","10,667",+173,"472,511","+8,631","217,614","2,300","4,802",73,"21,790,705","149,317","145,935,812",Europe
4,Peru,"309,278",,"10,952",,"200,938",,"97,388","1,265","9,378",332,"1,821,328","55,227","32,978,661",South America


In [7]:
df_bs.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']
df_bs = df_bs[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"331,039,330","3,097,538",+454,"133,991",+19,"1,355,524",+661,"1,608,023","15,371","9,357",405,"38,806,121","117,225"
1,Brazil,South America,"212,587,032","1,674,655",,"66,868",,"1,117,922","+45,693","489,865","8,318","7,878",315,"4,359,978","20,509"
2,India,Asia,"1,380,233,788","746,506","+3,025","20,684",+31,"458,618","+1,560","267,204","8,944",541,15,"10,473,771","7,588"
3,Russia,Europe,"145,935,812","700,792","+6,562","10,667",+173,"472,511","+8,631","217,614","2,300","4,802",73,"21,790,705","149,317"
4,Peru,South America,"32,978,661","309,278",,"10,952",,"200,938",,"97,388","1,265","9,378",332,"1,821,328","55,227"


In [8]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [9]:
df_bs['WHO Region'] = df_bs['Country/Region'].map(who_region)

df_bs[df_bs['WHO Region'].isna()]['Country/Region'].unique()

array(['French Guiana', 'CAR', 'Mayotte', 'Diamond Princess',
       'Channel Islands', 'Réunion', 'Isle of Man', 'Martinique',
       'Cayman Islands', 'Faeroe Islands', 'Guadeloupe', 'Gibraltar',
       'Brunei ', 'Aruba', 'Sint Maarten', 'French Polynesia',
       'Turks and Caicos', 'Macao', 'Saint Martin',
       'St. Vincent Grenadines', 'Curaçao', 'New Caledonia',
       'Falkland Islands', 'Montserrat', 'MS Zaandam'], dtype=object)

In [10]:
for col in df_bs.columns[2:]:
    df_bs[col] = df_bs[col].str.replace('[,+ ]', '', regex=True)
    df_bs[col] = df_bs[col].str.replace('N/A', '', regex=False)
df_bs = df_bs.replace('', np.nan)
df_bs = df_bs.fillna("Not Available")
df_bs = df_bs.astype(str)
df_bs.drop(columns=['NewCases','NewDeaths','NewRecovered','Serious,Critical'],axis=1,inplace=True)
df_bs.sort_values(by=["Country/Region"],inplace=True)
df_bs.drop(df_bs[df_bs["Continent"]=="Not Available"].index,axis=0,inplace=True)
df_bs = df_bs.reset_index(drop=True)

In [11]:
df_bs["Country/Region"][50]

'Djibouti'

In [12]:
df_bs.to_csv('worldometer_data.csv', index=False)

In [13]:
df=pd.read_csv('worldometer_data.csv')
df=df.set_index(df['Country/Region'])
df.head()

,Country/Region,Continent,Population,TotalCases,TotalDeaths,TotalRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
Country/Region,,,,,,,,,,,,
Afghanistan,Afghanistan,Asia,38937410,33594,936,20305,12353,863,24,77501,1990,EasternMediterranean
Albania,Albania,Europe,2877727,3106,83,1791,1232,1079,29,26637,9256,Europe
Algeria,Algeria,Africa,43861089,16879,968,12094,3817,385,22,Not Available,Not Available,Africa
Andorra,Andorra,Europe,77268,855,52,800,3,11065,673,3750,48532,Europe
Angola,Angola,Africa,32872107,386,21,117,248,12,0.6,10000,304,Africa


In [14]:
countries = pd.read_html("https://developers.google.com/public-data/docs/canonical/countries_csv")[0]
countries=countries.set_index(countries['name'])
countries.head()

,country,latitude,longitude,name
name,,,,
Andorra,AD,42.546245,1.601554,Andorra
United Arab Emirates,AE,23.424076,53.847818,United Arab Emirates
Afghanistan,AF,33.939110,67.709953,Afghanistan
Antigua and Barbuda,AG,17.060816,-61.796428,Antigua and Barbuda
Anguilla,AI,18.220554,-63.068615,Anguilla


In [15]:
lat,long=[],[]
m_lat,m_long=[],[]
for country in df['Country/Region']:
    try:
        lat.append((country,countries.loc[country.strip()]['latitude']))
    except:
        m_lat.append(country)
for country in df['Country/Region']:
    try:
        long.append((country,countries.loc[country.strip()]['longitude']))
    except:
        m_long.append(country)

m_lat

['CAR',
 'Cabo Verde',
 'Channel Islands',
 'Congo',
 'Curaçao',
 'Czechia',
 'DRC',
 'Eswatini',
 'Faeroe Islands',
 'Falkland Islands',
 'Ivory Coast',
 'Macao',
 'Myanmar',
 'North Macedonia',
 'Palestine',
 'S. Korea',
 'Saint Martin',
 'Sao Tome and Principe',
 'Sint Maarten',
 'South Sudan',
 'St. Vincent Grenadines',
 'Turks and Caicos',
 'UAE',
 'UK',
 'USA']

In [16]:
s=''
for i in m_lat:
    s+='("{}",),'.format(i)
print(s)

("CAR",),("Cabo Verde",),("Channel Islands",),("Congo",),("Curaçao",),("Czechia",),("DRC",),("Eswatini",),("Faeroe Islands",),("Falkland Islands",),("Ivory Coast",),("Macao",),("Myanmar",),("North Macedonia",),("Palestine",),("S. Korea",),("Saint Martin",),("Sao Tome and Principe",),("Sint Maarten",),("South Sudan",),("St. Vincent Grenadines",),("Turks and Caicos",),("UAE",),("UK",),("USA",),


In [17]:
lat+=[("CAR",6.111),("Cabo Verde",6.5388),("Channel Islands",49.372284),("Congo",-0.2280),("Curaçao",12.1696),("Czechia",49.8175),("DRC",-4.0383),("Eswatini",-26.5225),("Faeroe Islands",61.8926),("Falkland Islands",-51.7963),("Ivory Coast",7.5400),("Macao",22.1987),("Myanmar",21.9162),("North Macedonia",41.6086),("Palestine",31.9522),("S. Korea",35.9078),("Saint Martin",18.0826),("Sao Tome and Principe",0.1864),("Sint Maarten",18.0425),("South Sudan",6.8770),("St. Vincent Grenadines",12.9843),("Turks and Caicos",21.6940),("UAE",23.4241),("UK",55.3781),("USA",37.0902)]

long+=[("CAR",23.9394),("Cabo Verde",-23.0418),("Channel Islands",-2.364351),("Congo",15.8277),("Curaçao",-68.99),("Czechia",15.473),("DRC",-21.7587),("Eswatini",31.4659),("Faeroe Islands",-6.9118),("Falkland Islands",-59.5236),("Ivory Coast",5.5471),("Macao",113.5439),("Myanmar",95.956),("North Macedonia",21.7453),("Palestine",35.2332),("S. Korea",127.7669),("Saint Martin",63.0523),("Sao Tome and Principe",6.6131),("Sint Maarten",-63.0548),("South Sudan",31.307),("St. Vincent Grenadines",-61.2872),("Turks and Caicos",-71.7979),("UAE",53.8478),("UK",-3.436),("USA",-95.7129)]

In [18]:
del df["Country/Region"]
lat.sort()
long.sort()
df.sort_values("Country/Region",inplace=True)
df.head()

,Continent,Population,TotalCases,TotalDeaths,TotalRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
Country/Region,,,,,,,,,,,
Afghanistan,Asia,38937410,33594,936,20305,12353,863,24,77501,1990,EasternMediterranean
Albania,Europe,2877727,3106,83,1791,1232,1079,29,26637,9256,Europe
Algeria,Africa,43861089,16879,968,12094,3817,385,22,Not Available,Not Available,Africa
Andorra,Europe,77268,855,52,800,3,11065,673,3750,48532,Europe
Angola,Africa,32872107,386,21,117,248,12,0.6,10000,304,Africa


In [19]:
count=0
for i,j in zip([x[0] for x in lat],df.index):
    if i!=j:
        count+=1
print(count)

count1=0
for i,j in zip([x[0] for x in long],df.index):
    if i!=j:
        count1+=1
print(count1)

0
0


In [20]:
df['Latitude']=[x[1] for x in lat]
df['Longitude']=[y[1] for y in long]
df.tail()

,Continent,Population,TotalCases,TotalDeaths,TotalRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region,Latitude,Longitude
Country/Region,,,,,,,,,,,,,
Vietnam,Asia,97353756,369,Not Available,342,27,4,Not Available,275000,2825,WesternPacific,14.058324,108.277199
Western Sahara,Africa,597475,10,1,8,1,17,2,Not Available,Not Available,Africa,24.215527,-12.885834
Yemen,Asia,29832928,1297,348,591,358,43,12,120,4,EasternMediterranean,15.552727,48.516388
Zambia,Africa,18387764,1895,42,1348,505,103,2,56825,3090,Africa,-13.133897,27.849332
Zimbabwe,Africa,14866061,787,9,201,577,53,0.6,81335,5471,Africa,-19.015438,29.154857


In [21]:
df=df.reset_index()
df.head()

,Country/Region,Continent,Population,TotalCases,TotalDeaths,TotalRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region,Latitude,Longitude
0,Afghanistan,Asia,38937410,33594,936,20305,12353,863,24,77501,1990,EasternMediterranean,33.939110,67.709953
1,Albania,Europe,2877727,3106,83,1791,1232,1079,29,26637,9256,Europe,41.153332,20.168331
2,Algeria,Africa,43861089,16879,968,12094,3817,385,22,Not Available,Not Available,Africa,28.033886,1.659626
3,Andorra,Europe,77268,855,52,800,3,11065,673,3750,48532,Europe,42.546245,1.601554
4,Angola,Africa,32872107,386,21,117,248,12,0.6,10000,304,Africa,-11.202692,17.873887


In [32]:
map = folium.Map(location=[50,50], max_bounds = True, zoom_start=1.5,min_zoom=1,width='100%',height='100%')

for country,lat,long, cases, deaths, recovered, active, tests in zip(list(df["Country/Region"]),list(df['Latitude']),list(df['Longitude']),list(df["TotalCases"]),list(df["TotalDeaths"]),list(df["TotalRecovered"]),list(df["ActiveCases"]),list(df["TotalTests"])):
    folium.CircleMarker(location = [lat,long],
                       radius = 5,
                       color='red',
                       fill = True,
                       fill_color="red").add_to(map)
    folium.Marker(location = [lat,long],
                  popup=folium.Popup(('<strong><font color= blue>Country : '+country+'</font></striong><br>'+
                            '<strong>Total Cases : '+str(cases)+'</striong><br>'+
                            '<strong><font color = red>Total Deaths : '+str(deaths)+'</font></striong><br>'+
                            '<strong><font color = green>Total Recovered : '+str(recovered)+'</font></striong><br>'+
                            '<strong>Active Cases : '+str(active)+'</striong><br>'+
                            '<strong>Total Tests : '+str(tests)+'</striong><br>'),max_width=200)).add_to(map)
map